In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:

l2i_dic = { "O": 0,u'B-疾病': 1, u'B-药物': 2, u'B-手术': 3, u'B-症状': 4, u'B-预防措施': 5, u'B-观察方法': 6, u'B-诊断技术': 7,
           u'B-物理疗法': 8, u'B-护理技术': 9, u'B-病因': 10, u'B-病理': 11, u'B-医疗指标': 12,u'I-疾病': 13, u'I-药物': 14, u'I-手术': 15, u'I-症状': 16, u'I-预防措施': 17, u'I-观察方法': 18, u'I-诊断技术': 19,
           u'I-物理疗法': 20, u'I-护理技术': 21, u'I-病因': 22, u'I-病理': 23, u'I-医疗指标': 24,u'S-疾病': 25, u'S-药物': 26, u'S-手术': 27, u'S-症状': 28, u'S-预防措施': 29, u'S-观察方法': 30, u'S-诊断技术': 31,
           u'S-物理疗法': 32, u'S-护理技术': 33, u'S-病因': 34, u'S-病理': 35, u'S-医疗指标': 36,u'E-疾病': 37, u'E-药物': 38, u'E-手术': 39, u'E-症状': 40, u'E-预防措施': 41, u'E-观察方法': 42, u'E-诊断技术': 43,
           u'E-物理疗法': 44, u'E-护理技术': 45, u'E-病因': 46, u'E-病理': 47, u'E-医疗指标': 48,
           "<pad>": 49, "<start>": 50, "<eos>": 51}
i2l_dic = { 0:"O",1:u'B-疾病',2: u'B-药物',3: u'B-手术',4: u'B-症状',5: u'B-预防措施',6: u'B-观察方法',7: u'B-诊断技术',
           8:u'B-物理疗法',9: u'B-护理技术',10: u'B-病因',11: u'B-病理',12: u'B-医疗指标',13:u'I-疾病',14: u'I-药物',15: u'I-手术',16: u'I-症状',17: u'I-预防措施',18: u'I-观察方法',19: u'I-诊断技术',
           20:u'I-物理疗法',21: u'I-护理技术',22: u'I-病因',23: u'I-病理',24: u'I-医疗指标',25:u'S-疾病',26: u'S-药物',27: u'S-手术',28: u'S-症状',29: u'S-预防措施',30: u'S-观察方法',31: u'S-诊断技术',
           32:u'S-物理疗法',33: u'S-护理技术',34: u'S-病因',35: u'S-病理',36: u'S-医疗指标',37:u'E-疾病',38: u'E-药物',39: u'E-手术',40: u'E-症状',41: u'E-预防措施',42: u'E-观察方法',43: u'E-诊断技术',
           44:u'E-物理疗法',45: u'E-护理技术',46: u'E-病因',47: u'E-病理',48: u'E-医疗指标',
           49:"<pad>",50: "<start>",51: "<eos>"}
train_file = '/content/drive/My Drive/Colab Notebooks/bert-bilstm-crf-master/bert_lstm_crf_pytorch/data/train.txt'
dev_file = '/content/drive/My Drive/Colab Notebooks/bert-bilstm-crf-master/bert_lstm_crf_pytorch/data/dev.txt'
vocab_file = '/content/drive/My Drive/chinese_roberta_wwm_ext_pytorch.zip_files/vocab.txt'

save_model_dir =  '/content/drive/My Drive/Colab Notebooks/bert-bilstm-crf-master/bert_lstm_crf_pytorch/model'

max_length = 250
batch_size = 1
epochs = 3
tagset_size = len(l2i_dic)
use_cuda = True


In [ ]:
class InputFeatures(object):
    def __init__(self, text, label, input_id, label_id, input_mask):
        self.text = text
        self.label = label
        self.input_id = input_id
        self.label_id = label_id
        self.input_mask = input_mask

def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = {}
    index = 0
    with open(vocab_file, "r", encoding="utf-8") as reader:
        while True:
            token = reader.readline()
            if not token:
                break
            token = token.strip()
            vocab[token] = index
            index += 1
    return vocab

def load_file(file_path):
    contents = open(file_path, encoding='utf-8').readlines()
    text =[]
    label = []
    texts = []
    labels = []
    for line in contents:
        if line != '\n':
            line = line.strip().split('\t')
            text.append(line[0])
            label.append(line[-1])
        else:
            texts.append(text)
            labels.append(label)
            text = []
            label = []
    return texts, labels

def load_data(file_path, max_length, label_dic, vocab):
    texts, labels = load_file(file_path)
    assert len(texts) == len(labels)
    result = []
    for i in range(len(texts)):
        assert len(texts[i]) == len(labels[i])
        token = texts[i]
        label = labels[i]
        if len(token) > max_length-2:
            token = token[0:(max_length-2)]
            label = label[0:(max_length-2)]
        tokens_f =['[CLS]'] + token + ['[SEP]']
        label_f = ["<start>"] + label + ['<eos>']
        input_ids = [int(vocab[i]) if i in vocab else int(vocab['[UNK]']) for i in tokens_f]
        label_ids = [label_dic[i] for i in label_f]
        mask_bool=1
        input_mask = [mask_bool] * len(input_ids)
        while len(input_ids) < max_length:
            input_ids.append(0)
            input_mask.append(0)
            label_ids.append(label_dic['<pad>'])
        assert len(input_ids) == max_length
        assert len(input_mask) == max_length
        assert len(label_ids) == max_length
        feature = InputFeatures(text=tokens_f, label=label_f, input_id=input_ids, input_mask=input_mask,
                                label_id=label_ids)
        result.append(feature)
    return result

def recover_label(pred_var, gold_var, l2i_dic, i2l_dic):
    assert len(pred_var) == len(gold_var)
    pred_variable = []
    gold_variable = []
    for i in range(len(gold_var)):
     
        start_index = gold_var[i].index(l2i_dic['<start>'])
        end_index = gold_var[i].index(l2i_dic['<eos>'])
        pred_variable.append(pred_var[i][start_index:end_index])
        gold_variable.append(gold_var[i][start_index:end_index])

    pred_label = []
    gold_label = []
    for j in range(len(gold_variable)):
        pred_label.append([ i2l_dic[t] for t in pred_variable[j] ])
        gold_label.append([ i2l_dic[t] for t in gold_variable[j] ])

    return pred_label, gold_label

def get_ner_fmeasure(golden_lists, predict_lists, label_type="BMES"):
    sent_num = len(golden_lists)
    golden_full = []
    predict_full = []
    right_full = []
    right_tag = 0
    all_tag = 0
    for idx in range(0,sent_num):
        # word_list = sentence_lists[idx]
        golden_list = golden_lists[idx]
        predict_list = predict_lists[idx]
        for idy in range(len(golden_list)):
            if golden_list[idy] == predict_list[idy]:
                right_tag += 1
        all_tag += len(golden_list)
        if label_type == "BMES":
            gold_matrix = get_ner_BMES(golden_list)
            pred_matrix = get_ner_BMES(predict_list)
        else:
            gold_matrix = get_ner_BIO(golden_list)
            pred_matrix = get_ner_BIO(predict_list)
        right_ner = list(set(gold_matrix).intersection(set(pred_matrix)))
        golden_full += gold_matrix
        predict_full += pred_matrix
        right_full += right_ner
    right_num = len(right_full)
    golden_num = len(golden_full)
    predict_num = len(predict_full)
    if predict_num == 0:
        precision = -1
    else:
        precision =  (right_num+0.0)/predict_num
    if golden_num == 0:
        recall = -1
    else:
        recall = (right_num+0.0)/golden_num
    if (precision == -1) or (recall == -1) or (precision+recall) <= 0.:
        f_measure = -1
    else:
        f_measure = 2*precision*recall/(precision+recall)
    accuracy = (right_tag+0.0)/all_tag
    # print "Accuracy: ", right_tag,"/",all_tag,"=",accuracy
    print ("gold_num = ", golden_num, " pred_num = ", predict_num, " right_num = ", right_num)
    return accuracy, precision, recall, f_measure

def reverse_style(input_string):
    target_position = input_string.index('[')
    input_len = len(input_string)
    output_string = input_string[target_position:input_len] + input_string[0:target_position]
    return output_string

def get_ner_BMES(label_list):
    # list_len = len(word_list)
    # assert(list_len == len(label_list)), "word list size unmatch with label list"
    list_len = len(label_list)
    begin_label = 'B-'
    end_label = 'E-'
    single_label = 'S-'
    whole_tag = ''
    index_tag = ''
    tag_list = []
    stand_matrix = []
    for i in range(0, list_len):
        # wordlabel = word_list[i]
        current_label = label_list[i].upper()
        if begin_label in current_label:
            if index_tag != '':
                tag_list.append(whole_tag + ',' + str(i - 1))
            whole_tag = current_label.replace(begin_label, "", 1) + '[' + str(i)
            index_tag = current_label.replace(begin_label, "", 1)

        elif single_label in current_label:
            if index_tag != '':
                tag_list.append(whole_tag + ',' + str(i - 1))
            whole_tag = current_label.replace(single_label, "", 1) + '[' + str(i)
            tag_list.append(whole_tag)
            whole_tag = ""
            index_tag = ""
        elif end_label in current_label:
            if index_tag != '':
                tag_list.append(whole_tag + ',' + str(i))
            whole_tag = ''
            index_tag = ''
        else:
            continue
    if (whole_tag != '') & (index_tag != ''):
        tag_list.append(whole_tag)
    tag_list_len = len(tag_list)

    for i in range(0, tag_list_len):
        if len(tag_list[i]) > 0:
            tag_list[i] = tag_list[i] + ']'
            insert_list = reverse_style(tag_list[i])
            stand_matrix.append(insert_list)
    # print stand_matrix
    return stand_matrix


def save_model(path, model, epoch):
    pass

def load_model(path, model):
    return model


if __name__ =="__main__":
    pass





In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import os
def log_sum_exp(vec, m_size):
    """
    Args:
        vec: size=(batch_size, vanishing_dim, hidden_dim)
        m_size: hidden_dim

    Returns:
        size=(batch_size, hidden_dim)
    """
    _, idx = torch.max(vec, 1)  # B * 1 * M
    max_score = torch.gather(vec, 1, idx.view(-1, 1, m_size)).view(-1, 1, m_size)  # B * M
    return max_score.view(-1, m_size) + torch.log(torch.sum(
        torch.exp(vec - max_score.expand_as(vec)), 1)).view(-1, m_size)


class CRF(nn.Module):

    def __init__(self, **kwargs):
        """
        Args:
            target_size: int, target size
            use_cuda: bool, 鏄惁浣跨敤gpu, default is True
            average_batch: bool, loss鏄惁浣滃钩鍧ꣲ default is True
        """
        super(CRF, self).__init__()
        for k in kwargs:
            self.__setattr__(k, kwargs[k])
        self.START_TAG_IDX, self.END_TAG_IDX = -2, -1
        init_transitions = torch.zeros(self.target_size+2, self.target_size+2)
        init_transitions[:, self.START_TAG_IDX] = -1000.
        init_transitions[self.END_TAG_IDX, :] = -1000.
        if self.use_cuda:
            init_transitions = init_transitions.cuda()
        self.transitions = nn.Parameter(init_transitions)
    def _forward_alg(self, feats, mask=None):
        """
        Do the forward algorithm to compute the partition function (batched).

        Args:
            feats: size=(batch_size, seq_len, self.target_size+2)
            mask: size=(batch_size, seq_len)

        Returns:
            xxx
        """
        batch_size = feats.size(0)
        seq_len = feats.size(1)
        tag_size = feats.size(-1)

        mask = mask.transpose(1, 0).contiguous()
        ins_num = batch_size * seq_len
        feats = feats.transpose(1, 0).contiguous().view(
            ins_num, 1, tag_size).expand(ins_num, tag_size, tag_size)

        scores = feats + self.transitions.view(
            1, tag_size, tag_size).expand(ins_num, tag_size, tag_size)
        scores = scores.view(seq_len, batch_size, tag_size, tag_size)
        seq_iter = enumerate(scores)
        try:
            _, inivalues = seq_iter.__next__()
        except:
            _, inivalues = seq_iter.next()

        partition = inivalues[:, self.START_TAG_IDX, :].clone().view(batch_size, tag_size, 1)
        for idx, cur_values in seq_iter:
            cur_values = cur_values + partition.contiguous().view(
                batch_size, tag_size, 1).expand(batch_size, tag_size, tag_size)
            cur_partition = log_sum_exp(cur_values, tag_size)
            mask_idx = mask[idx, :].view(batch_size, 1).expand(batch_size, tag_size)
            masked_cur_partition = cur_partition.masked_select(mask_idx.byte())
            if masked_cur_partition.dim() != 0:
                mask_idx = mask_idx.contiguous().view(batch_size, tag_size, 1)
                partition.masked_scatter_(mask_idx.byte(), masked_cur_partition)
        cur_values = self.transitions.view(1, tag_size, tag_size).expand(
            batch_size, tag_size, tag_size) + partition.contiguous().view(
                batch_size, tag_size, 1).expand(batch_size, tag_size, tag_size)
        cur_partition = log_sum_exp(cur_values, tag_size)
        final_partition = cur_partition[:, self.END_TAG_IDX]
        return final_partition.sum(), scores
        
    def _viterbi_decode(self, feats, mask=None):
        """
        Args:
            feats: size=(batch_size, seq_len, self.target_size+2)
            mask: size=(batch_size, seq_len)

        Returns:
            decode_idx: (batch_size, seq_len), viterbi decode缁撴灉
            path_score: size=(batch_size, 1), 姣忎釜鍙ュ瓙鐨勫緱鍒ꓱ        """
        batch_size = feats.size(0)
        seq_len = feats.size(1)
        tag_size = feats.size(-1)

        length_mask = torch.sum(mask, dim=1).view(batch_size, 1).long()
        mask = mask.transpose(1, 0).contiguous()
        ins_num = seq_len * batch_size
        feats = feats.transpose(1, 0).contiguous().view(
            ins_num, 1, tag_size).expand(ins_num, tag_size, tag_size)

        scores = feats + self.transitions.view(
            1, tag_size, tag_size).expand(ins_num, tag_size, tag_size)
        scores = scores.view(seq_len, batch_size, tag_size, tag_size)

        seq_iter = enumerate(scores)
        # record the position of the best score
        back_points = list()
        partition_history = list()
        mask = (1 - mask.long()).byte()
        try:
            _, inivalues = seq_iter.__next__()
        except:
            _, inivalues = seq_iter.next()
        partition = inivalues[:, self.START_TAG_IDX, :].clone().view(batch_size, tag_size, 1)
        partition_history.append(partition)

        for idx, cur_values in seq_iter:
            cur_values = cur_values + partition.contiguous().view(
                batch_size, tag_size, 1).expand(batch_size, tag_size, tag_size)
            partition, cur_bp = torch.max(cur_values, 1)
            partition_history.append(partition.unsqueeze(-1))

            cur_bp.masked_fill_(mask[idx].view(batch_size, 1).expand(batch_size, tag_size), 0)
            back_points.append(cur_bp)

        partition_history = torch.cat(partition_history).view(
            seq_len, batch_size, -1).transpose(1, 0).contiguous()

        last_position = length_mask.view(batch_size, 1, 1).expand(batch_size, 1, tag_size) - 1
        last_partition = torch.gather(
            partition_history, 1, last_position).view(batch_size, tag_size, 1)

        last_values = last_partition.expand(batch_size, tag_size, tag_size) + \
            self.transitions.view(1, tag_size, tag_size).expand(batch_size, tag_size, tag_size)
        _, last_bp = torch.max(last_values, 1)
        pad_zero = Variable(torch.zeros(batch_size, tag_size)).long()
        if self.use_cuda:
            pad_zero = pad_zero.cuda()
        back_points.append(pad_zero)
        back_points = torch.cat(back_points).view(seq_len, batch_size, tag_size)

        pointer = last_bp[:, self.END_TAG_IDX]
        insert_last = pointer.contiguous().view(batch_size, 1, 1).expand(batch_size, 1, tag_size)
        back_points = back_points.transpose(1, 0).contiguous()

        back_points.scatter_(1, last_position, insert_last)

        back_points = back_points.transpose(1, 0).contiguous()

        decode_idx = Variable(torch.LongTensor(seq_len, batch_size))
        if self.use_cuda:
            decode_idx = decode_idx.cuda()
        decode_idx[-1] = pointer.data
        for idx in range(len(back_points)-2, -1, -1):
            pointer = torch.gather(back_points[idx], 1, pointer.contiguous().view(batch_size, 1))
            decode_idx[idx] = pointer.view(-1).data
        path_score = None
        decode_idx = decode_idx.transpose(1, 0)
        return path_score, decode_idx

    def forward(self, feats, mask=None):
        path_score, best_path = self._viterbi_decode(feats, mask)
        return path_score, best_path

    def _score_sentence(self, scores, mask, tags):
        """
        Args:
            scores: size=(seq_len, batch_size, tag_size, tag_size)
            mask: size=(batch_size, seq_len)
            tags: size=(batch_size, seq_len)

        Returns:
            score:
        """
        batch_size = scores.size(1)
        seq_len = scores.size(0)
        tag_size = scores.size(-1)

        new_tags = Variable(torch.LongTensor(batch_size, seq_len))
        if self.use_cuda:
            new_tags = new_tags.cuda()
        for idx in range(seq_len):
            if idx == 0:
                new_tags[:, 0] = (tag_size - 2) * tag_size + tags[:, 0]
            else:
                new_tags[:, idx] = tags[:, idx-1] * tag_size + tags[:, idx]

        end_transition = self.transitions[:, self.END_TAG_IDX].contiguous().view(
            1, tag_size).expand(batch_size, tag_size)
        length_mask = torch.sum(mask, dim=1).view(batch_size, 1).long()
        end_ids = torch.gather(tags, 1, length_mask-1)

        end_energy = torch.gather(end_transition, 1, end_ids)

        new_tags = new_tags.transpose(1, 0).contiguous().view(seq_len, batch_size, 1)
        tg_energy = torch.gather(scores.view(seq_len, batch_size, -1), 2, new_tags).view(
            seq_len, batch_size)
        tg_energy = tg_energy.masked_select(mask.transpose(1, 0))

        gold_score = tg_energy.sum() + end_energy.sum()

        return gold_score

    def neg_log_likelihood_loss(self, feats, mask, tags):
        """
        Args:
            feats: size=(batch_size, seq_len, tag_size)
            mask: size=(batch_size, seq_len)
            tags: size=(batch_size, seq_len)
        """
        batch_size = feats.size(0)
        mask = mask.byte()
        forward_score, scores = self._forward_alg(feats, mask)
        gold_score = self._score_sentence(scores, mask, tags)
        if self.average_batch:
            return (forward_score - gold_score) / batch_size
        return forward_score - gold_score







In [ ]:
! pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 2.8MB/s 


In [ ]:



import torch.nn as nn
from pytorch_pretrained_bert import BertModel
from torch.autograd import Variable
import torch

class BERT_LSTM_CRF(nn.Module):
    def __init__(self, bert_config, tagset_size, embedding_dim, hidden_dim, rnn_layers, dropout_ratio, dropout1, use_cuda):
        super(BERT_LSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.word_embeds = BertModel.from_pretrained(bert_config)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,
                            num_layers=rnn_layers, bidirectional=True, dropout=dropout_ratio, batch_first=True)
        self.rnn_layers = rnn_layers
        self.dropout1 = nn.Dropout(p=dropout1)
        self.crf = CRF(target_size=tagset_size, average_batch=True, use_cuda=use_cuda)
        self.liner = nn.Linear(hidden_dim*2, tagset_size+2)
        self.tagset_size = tagset_size
        self.use_cuda =  use_cuda
    def rand_init_hidden(self, batch_size):
        if self.use_cuda:
            return Variable(
                torch.randn(2 * self.rnn_layers, batch_size, self.hidden_dim)).cuda(), Variable(
                torch.randn(2 * self.rnn_layers, batch_size, self.hidden_dim)).cuda()
        else:
            return Variable(
                torch.randn(2 * self.rnn_layers, batch_size, self.hidden_dim)), Variable(
                torch.randn(2 * self.rnn_layers, batch_size, self.hidden_dim))

    def get_output_score(self, sentence, attention_mask=None):
        batch_size = sentence.size(0)
        seq_length = sentence.size(1)
        embeds, _ = self.word_embeds(sentence, attention_mask=attention_mask, output_all_encoded_layers=False)
        hidden = self.rand_init_hidden(batch_size)
        # if embeds.is_cuda:
        #     hidden = (i.cuda() for i in hidden)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim * 2)
        d_lstm_out = self.dropout1(lstm_out)
        l_out = self.liner(d_lstm_out)
        lstm_feats = l_out.contiguous().view(batch_size, seq_length, -1)
        return lstm_feats

    def forward(self, sentence, masks):
        lstm_feats = self.get_output_score(sentence)
        scores, tag_seq = self.crf._viterbi_decode(lstm_feats, masks.byte())
        return tag_seq

    def neg_log_likelihood_loss(self, sentence, mask, tags):
        lstm_feats = self.get_output_score(sentence)
        loss_value = self.crf.neg_log_likelihood_loss(lstm_feats, mask, tags)
        batch_size = lstm_feats.size(0)
        loss_value /= float(batch_size)
        return loss_value







In [ ]:

import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
if torch.cuda.is_available():
    device = torch.device("cuda", 0)
    print('device',device)
    use_cuda = True
else:
    device = torch.device("cpu")
    use_cuda = False
vocab = load_vocab(vocab_file)
####璇诲彇璁粌闆?
print('max_length',max_length)
train_data = load_data(train_file, max_length=max_length, label_dic=l2i_dic, vocab=vocab)
train_ids = torch.LongTensor([temp.input_id for temp in train_data])
train_masks = torch.LongTensor([temp.input_mask for temp in train_data])
train_tags = torch.LongTensor([temp.label_id for temp in train_data])
train_dataset = TensorDataset(train_ids, train_masks, train_tags)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
#######璇诲彇娴嬭瘯闆?
dev_data = load_data(dev_file, max_length=max_length, label_dic=l2i_dic, vocab=vocab)
dev_ids = torch.LongTensor([temp.input_id for temp in dev_data])
dev_masks = torch.LongTensor([temp.input_mask for temp in dev_data])
dev_tags = torch.LongTensor([temp.label_id for temp in dev_data])
dev_dataset = TensorDataset(dev_ids, dev_masks, dev_tags)
dev_loader = DataLoader(dev_dataset, shuffle=True, batch_size=batch_size)
######娴嬭瘯鍑芥暟
def evaluate(medel, dev_loader):
    medel.eval()
    pred = []
    gold = []
    print('evaluate')
    for i, dev_batch in enumerate(dev_loader):
        sentence, masks, tags = dev_batch
        sentence, masks, tags = Variable(sentence), Variable(masks), Variable(tags)
        if use_cuda:
            sentence = sentence.cuda()
            masks = masks.cuda()
            tags = tags.cuda()
        predict_tags = medel(sentence, masks)
        pred.extend([t for t in predict_tags.tolist()])
        gold.extend([t for t in tags.tolist()])
    pred_label,gold_label = recover_label(pred, gold, l2i_dic,i2l_dic)
    acc, p, r, f = get_ner_fmeasure(gold_label,pred_label)
    print(get_right_tag(gold_label,pred_label))
    print('p: {}锛宺: {}, f: {}'.format(p, r, f))
    model.train()
    return acc, p, r, f
########鍔犺浇妯″瀷
model = BERT_LSTM_CRF('/content/drive/My Drive/chinese_roberta_wwm_ext_pytorch.zip_files', tagset_size, 768, 250, 1,
                      dropout_ratio=0.4, dropout1=0.4, use_cuda = use_cuda)
if use_cuda:
    model.cuda()
model.train()
optimizer = getattr(optim, 'Adam')
optimizer = optimizer(model.parameters(), lr=0.00003, weight_decay=0.00005)
best_f = -100
for epoch in range(epochs):
    print('epoch: {}锛宼rain'.format(epoch))
    for i, train_batch in enumerate(tqdm(train_loader)):
        model.zero_grad()
        sentence, masks, tags = train_batch
        sentence, masks, tags = Variable(sentence), Variable(masks), Variable(tags)
        if use_cuda:
            sentence = sentence.cuda()
            masks = masks.cuda()
            tags = tags.cuda()
        loss = model.neg_log_likelihood_loss(sentence, masks, tags)
        loss.backward()
        optimizer.step()
    print('epoch: {}锛宭oss: {}'.format(epoch, loss.item()))
    acc, p, r, f = evaluate(model,dev_loader)
    if f > best_f:
        model_name = save_model_dir + '.' + str(epochs) + ".pkl"
        torch.save(model.state_dict(), model_name)
        best_f = f

device cuda:0
max_length 250


  0%|          | 0/1368 [00:00<?, ?it/s]

epoch: 0锛宼rain


100%|██████████| 1368/1368 [19:35<00:00,  1.16it/s]


epoch: 0锛宭oss: 69.65026092529297
evaluate
gold_num =  1421  pred_num =  0  right_num =  0


ValueError: ignored

In [ ]:
dev_loader.size()

In [ ]:
import torch
model =BERT_LSTM_CRF('/content/drive/My Drive/chinese_roberta_wwm_ext_pytorch.zip_files', tagset_size, 768, 200, 1,
                      dropout_ratio=0.4, dropout1=0.4, use_cuda = use_cuda)
model.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/bert-bilstm-crf-master/bert_lstm_crf_pytorch/model.100.pkl'))
model.cuda()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


BERT_LSTM_CRF(
  (word_embeds): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            

预测

In [ ]:
f=open("/content/drive/My Drive/Colab Notebooks/kashgari/kashgari/data0/data_unlabeled/abstract_012.txt","r")
for line in f:
    line=f.read()
    line=line.replace("\n","")
    raw=line.split("。")

In [ ]:

pre_data=[]
for i in raw:
  sen=[]
  for j in i:
    sen.append(j)
  pre_data.append(sen)

In [ ]:
from torch.utils.data import TensorDataset
vocab = load_vocab(vocab_file)
pre_ids = []
pre_masks=[]
for i in range(len(pre_data)):
    token = pre_data[i]
    if len(token) > max_length-2:
        token = token[0:(max_length-2)]
    tokens_f =['[CLS]'] + token + ['[SEP]']
    input_ids = [int(vocab[i]) if i in vocab else int(vocab['[UNK]']) for i in tokens_f]
    mask_bool=1
    input_mask = [mask_bool] * len(input_ids)
    while len(input_ids) < max_length:
        input_ids.append(0)
        input_mask.append(0)
    assert len(input_ids) == max_length
    assert len(input_mask) == max_length
    pre_ids.append(input_ids)
    pre_masks.append(input_mask)
pre_ids_f=torch.LongTensor(pre_ids)
pre_masks_f=torch.LongTensor(pre_masks)

In [ ]:

from torch.utils.data import DataLoader
pre_dataset=TensorDataset(pre_ids_f, pre_masks_f)
pre_loader = DataLoader(pre_dataset, shuffle=False, batch_size=12)

In [ ]:

import warnings
warnings.filterwarnings('ignore')
pred=[]
from torch.autograd import Variable
model.eval()
entities_list=[]
type_list=[]
for i, pre_batch in enumerate(pre_loader):
    sentence, masks = pre_batch
    sentence, masks = Variable(sentence), Variable(masks)
    if use_cuda:
        sentence = sentence.cuda()
        masks = masks.cuda()
    predict_tags = model(sentence, masks)
    for i in range(len(predict_tags)):
        tp=[]
        types=[]
        entities=[]
        entity=[]
        for j in range(len(predict_tags[i])):
            if (predict_tags[i][j] !=0 and predict_tags[i][j] !=50 and predict_tags[i][j] !=51) and (predict_tags[i][j]>=predict_tags[i][j-1]):
                entity.append(j)
                tp.append(i2l_dic[predict_tags.tolist()[i][j]])
            else:
                if entity !=[]:
                    entities.append(entity)
                    types.append(tp)
                entity=[]
                tp=[]
        entities_list.append(entities)
        type_list.append(types)

In [ ]:
recovered_entities=[]
for i in  range(len(entities_list)):
    recovered_word=[]
    for j in range(len(entities_list[i])):
        recover_char=""
        for m in range(len(entities_list[i][j])):
            try:
                recover_char+=pre_data[i][entities_list[i][j][m]-1]
            except:
                pass
        recovered_word.append(recover_char)
    recovered_entities.append(recovered_word)

In [ ]:
entities_type=[]
for i in range(len(recovered_entities)):
    entity_type=[]
    for j in range(len(recovered_entities[i])):
        enti_dic={}
        enti_dic={type_list[i][j][0][2:]:recovered_entities[i][j]}
        entity_type.append(enti_dic)
    entities_type.append(entity_type)

In [ ]:
entities_sen=list(zip(raw,entities_type))

In [ ]:
rel_sen=[]
for i in entities_sen:
    for j in i[1]:
        for k in i[1]:
            if i[1].index(j)<=i[1].index(k):
                rel_sen.append((list(j.keys())[0],list(j.values())[0],list(k.keys())[0],list(k.values())[0],i[0]))

In [ ]:
import pandas as pd
df=pd.DataFrame(rel_sen)

In [ ]:
df.to_csv("/content/drive/My Drive/Colab Notebooks/bert-bilstm-crf-master/bert_lstm_crf_pytorch/data/12.txt")

In [ ]:
import os
import pandas as pd
from torch.utils.data import DataLoader
import torch
import warnings
from torch.utils.data import TensorDataset
warnings.filterwarnings('ignore')
str_file_nums=[]
file_nums=range(12,153)
for i in file_nums:
  if i<100:
    str_file_nums.append("0"+str(i))
  else:
    str_file_nums.append(str(i))
for num in range(len(str_file_nums)):
  file="/content/drive/My Drive/Colab Notebooks/kashgari/kashgari/data0/data_unlabeled/abstract_"+str_file_nums[num]+".txt"
  with open(file,encoding="utf8") as f:
    for line in f:
        line=f.read()
        line=line.replace("\n","")
        raw=line.split("。")
    pre_data=[]
    for i in raw:
      sen=[]
      for j in i:
        sen.append(j)
      pre_data.append(sen)
    vocab = load_vocab(vocab_file)
    pre_ids = []
    pre_masks=[]
    for i in range(len(pre_data)):
        token = pre_data[i]
        if len(token) > max_length-2:
            token = token[0:(max_length-2)]
        tokens_f =['[CLS]'] + token + ['[SEP]']
        input_ids = [int(vocab[i]) if i in vocab else int(vocab['[UNK]']) for i in tokens_f]
        mask_bool=1
        input_mask = [mask_bool] * len(input_ids)
        while len(input_ids) < max_length:
            input_ids.append(0)
            input_mask.append(0)
        assert len(input_ids) == max_length
        assert len(input_mask) == max_length
        pre_ids.append(input_ids)
        pre_masks.append(input_mask)
    pre_ids_f=torch.LongTensor(pre_ids)
    pre_masks_f=torch.LongTensor(pre_masks)
    pre_dataset=TensorDataset(pre_ids_f, pre_masks_f)
    pre_loader = DataLoader(pre_dataset, shuffle=False, batch_size=12)
    pred=[]
    from torch.autograd import Variable
    model.eval()
    entities_list=[]
    type_list=[]
    for i, pre_batch in enumerate(pre_loader):
        sentence, masks = pre_batch
        sentence, masks = Variable(sentence), Variable(masks)
        if use_cuda:
            sentence = sentence.cuda()
            masks = masks.cuda()
        predict_tags = model(sentence, masks)
        for i in range(len(predict_tags)):
            tp=[]
            types=[]
            entities=[]
            entity=[]
            for j in range(len(predict_tags[i])):
                if (predict_tags[i][j] !=0 and predict_tags[i][j] !=50 and predict_tags[i][j] !=51) and (predict_tags[i][j]>=predict_tags[i][j-1]):
                    entity.append(j)
                    tp.append(i2l_dic[predict_tags.tolist()[i][j]])
                else:
                    if entity !=[]:
                        entities.append(entity)
                        types.append(tp)
                    entity=[]
                    tp=[]
            entities_list.append(entities)
            type_list.append(types)
    recovered_entities=[]
    for i in  range(len(entities_list)):
        recovered_word=[]
        for j in range(len(entities_list[i])):
            recover_char=""
            for m in range(len(entities_list[i][j])):
                try:
                    recover_char+=pre_data[i][entities_list[i][j][m]-1]
                except:
                    pass
            recovered_word.append(recover_char)
        recovered_entities.append(recovered_word)
    entities_type=[]
    for i in range(len(recovered_entities)):
        entity_type=[]
        for j in range(len(recovered_entities[i])):
            enti_dic={}
            enti_dic={type_list[i][j][0][2:]:recovered_entities[i][j]}
            entity_type.append(enti_dic)
        entities_type.append(entity_type)
    entities_sen=list(zip(raw,entities_type))
    rel_sen=[]
    for i in entities_sen:
        for j in i[1]:
            for k in i[1]:
                if i[1].index(j)<i[1].index(k):
                    rel_sen.append((list(j.keys())[0],list(j.values())[0],list(k.keys())[0],list(k.values())[0],i[0]))
    df=pd.DataFrame(rel_sen)
    df.to_csv("/content/drive/My Drive/Colab Notebooks/bert-bilstm-crf-master/bert_lstm_crf_pytorch/data/"+str_file_nums[num]+".txt")

In [ ]:
from sklearn.metrics import classification_report
def get_right_tag(golden_lists, predict_lists, label_type="BMES"):
    sent_num = len(golden_lists)
    golden_full = []
    predict_full = []
    right_full = []
    right_tag = 0
    all_tag = 0
    for idx in range(0,sent_num):
        # word_list = sentence_lists[idx]
        golden_list = golden_lists[idx]
        predict_list = predict_lists[idx]
        for idy in range(len(golden_list)):
            if golden_list[idy] == predict_list[idy]:
                right_tag += 1
        all_tag += len(golden_list)
        if label_type == "BMES":
            gold_matrix = get_ner_BMES(golden_list)
            pred_matrix = get_ner_BMES(predict_list)
        else:
            gold_matrix = get_ner_BIO(golden_list)
            pred_matrix = get_ner_BIO(predict_list)
        right_ner = list(set(gold_matrix).intersection(set(pred_matrix)))
        golden_full += gold_matrix
        predict_full += pred_matrix
        right_full += right_ner
    right_num = len(right_full)
    golden_num = len(golden_full)
    predict_num = len(predict_full)
    if predict_num == 0:
        precision = -1
    else:
        precision =  (right_num+0.0)/predict_num
    if golden_num == 0:
        recall = -1
    else:
        recall = (right_num+0.0)/golden_num
    if (precision == -1) or (recall == -1) or (precision+recall) <= 0.:
        f_measure = -1
    else:
        f_measure = 2*precision*recall/(precision+recall)
    accuracy = (right_tag+0.0)/all_tag
    print(classification_report(golden_full,predict_full))
    